In [48]:
import pandas as pd

df = pd.read_csv('evaluation_resultsmixtral8x7b.csv')

df.head()

,prompt,true_label,predicted_label
0,Basado en la oración: 'El desagüe se ha atasca...,True,False.\n\nLa oración correcta de
1,Basado en la oración: 'Jane llamó a la puerta ...,False,True\n\nLa oración 'Jane llam
2,Basado en la oración: 'Beth no se enfadó con S...,True,"False.\n\nThe phrase ""se esperó"
3,Basado en la oración: 'Nadie entra en Facebook...,True,"False.\n\nThe original sentence states that """
4,Basado en la oración: 'El hombre no pudo levan...,False,True\n\nThe sentence 'El hijo pes


In [49]:
df.describe()

,prompt,true_label,predicted_label
count,71,71,71
unique,70,2,70
top,Basado en la oración: 'Estaban retransmitiendo...,False,False.\n\nLa oración correcta de
freq,2,40,2


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prompt           71 non-null     object
 1   true_label       71 non-null     bool  
 2   predicted_label  71 non-null     object
dtypes: bool(1), object(2)
memory usage: 1.3+ KB


In [51]:
df = df.loc[:, ["true_label", "predicted_label"]]

df['predicted_label'] = df['predicted_label'].apply(lambda x: x.lower().strip())

df.head()

,true_label,predicted_label
0,True,false.\n\nla oración correcta de
1,False,true\n\nla oración 'jane llam
2,True,"false.\n\nthe phrase ""se esperó"
3,True,"false.\n\nthe original sentence states that """
4,False,true\n\nthe sentence 'el hijo pes


In [52]:
import re
def extract_boolean(text):
    pattern = re.compile(r'\btrue\b', re.IGNORECASE)
    if pattern.search(text):
        return True
    pattern = re.compile(r'\bfalse\b', re.IGNORECASE)
    if pattern.search(text):
        return False
    return None

# Aplicar la función al dataframe
df['predicted_label'] = df['predicted_label'].apply(extract_boolean)
df['predicted_label'] = df['predicted_label'].astype(bool)

df.head()

,true_label,predicted_label
0,True,False
1,False,True
2,True,False
3,True,False
4,False,True


In [53]:
df.isna().sum()

true_label         0
predicted_label    0
dtype: int64

In [54]:
df.dropna(inplace=True)

In [55]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

true_labels = df['true_label']
predicted_labels = df['predicted_label']

accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

accuracy, recall, f1


(0.49295774647887325, 0.3870967741935484, 0.4)

In [69]:
import jsonlines
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, f1_score

# Load JSONL file into a DataFrame
data = []
with jsonlines.open('claude-v2.jsonl') as reader:
    for obj in reader:
        prompt = obj['inputRecord']['prompt']
        reference_response = obj['inputRecord']['referenceResponse'].lower().strip()  # Convert to lower and strip whitespaces
        model_response = obj['modelResponses'][0]['response'].lower().strip()  # Convert to lower and strip whitespaces
        
        # Append only if both reference_response and model_response are not missing
        if reference_response and model_response:
            data.append({'prompt': prompt, 'reference_response': reference_response, 'model_response': model_response})

df = pd.DataFrame(data)

# Function to map responses, handling unexpected or malformed inputs
def map_response(response):
    response = response.strip().lower()  # Clean and convert to lowercase
    pattern = re.compile(r'\btrue\b', re.IGNORECASE)
    if pattern.search(response):
        return True
    pattern = re.compile(r'\bfalse\b', re.IGNORECASE)
    if pattern.search(response):
        return False
    return None

# Apply the mapping function
df['reference_response'] = df['reference_response'].apply(map_response)
df['model_response'] = df['model_response'].apply(map_response)

# Drop rows where either response is None (i.e., was not 'true' or 'false')
df.dropna(subset=['reference_response', 'model_response'], inplace=True)

# Ensure types are boolean after dropping NAs
df['reference_response'] = df['reference_response'].astype(bool)
df['model_response'] = df['model_response'].astype(bool)

# Calculate metrics
accuracy = accuracy_score(df['reference_response'], df['model_response'])
recall = recall_score(df['reference_response'], df['model_response'])
f1 = f1_score(df['reference_response'], df['model_response'])

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Accuracy: 0.43661971830985913
Recall: 0.22580645161290322
F1 Score: 0.25925925925925924
